In [1]:
import pandas as pd
import bs4
import requests
import json
import re
import tqdm

import scipy.stats as st
import numpy as np


In [2]:
# As of 2020 https://www.ncbi.nlm.nih.gov/research/cog/
COG_dict = {
    "A" : "RNA processing and modification",
    "B" : "Chromatin structure and dynamics",
    "C" : "Energy production and conversion",
    "D" : "Cell cycle control, cell division, chromosome partitioning",
    "E" : "Amino acid transport and metabolism",
    "F" : "Nucleotide transport and metabolism",
    "G" : "Carbohydrate transport and metabolism",
    "H" : "Coenzyme transport and metabolism",
    "I" : "Lipid transport and metabolism",
    "J" : "Translation, ribosomal structure and biogenesis",
    "K" : "Transcription",
    "L" : "Replication, recombination, and repair",
    "M" : "Cell wall/membrane/envelope biogenesis",
    "N" : "Cell motility",
    "O" : "Posttranslational modification, protein turnover, chaperones",
    "P" : "Inorganic ion transport and metabolism",
    "Q" : "Secondary metabolites biosynthesis, transport and catabolism",
    "T" : "Signal transduction mechanisms",
    "U" : "Intracellular trafficking, secretion, and vesicular transport",
    "V" : "Defense mechanisms",
    "W" : "Extracellular structures",
    "X" : "Mobilome: prophages, transposons",
    "Y" : "Nuclear structure",
    "Z" : "Cytoskeleton",
    "R" : "General function prediction only",
    "S" : "Function unknown",
}

In [3]:
def term_count_dict_from_annotation_df(annot_df, term_column):
    
    column = annot_df[term_column].values
    
    funct_terms = []
    for entry in column:
        if entry != '-':
            if term_column == 'COG_category':
                # terms = [f'{e}: {COG_dict[e]}' for e in entry]
                terms = list(entry)
            else:
                terms = entry.split(',')
            for t in terms:
                funct_terms.append(t)

#     len(terms)
    
    term_count_dict = {}
    
    for t in funct_terms:
        count = term_count_dict.get(t, 0)
        count += 1
        term_count_dict[t] = count
        
    return term_count_dict

In [4]:
def enrichment_analysis(module, leiden_label_df, phases, background_annotation, term_column):
    
    module_ttids = leiden_label_df.loc[leiden_label_df[f'leiden_label_{phases}'] == module]['TTHERM_ID'].values
    
    module_annotation = background_annotation.loc[background_annotation['TTHERM_ID'].isin(module_ttids)]
    
    background_term_dict = term_count_dict_from_annotation_df(background_annotation, term_column)
    module_term_dict = term_count_dict_from_annotation_df(module_annotation, term_column)
    
    bs = []
    ps = []
    folds = []
    terms = []
    
    for t, module_count in module_term_dict.items():
        
        background_count = background_term_dict[t]
        module_size = len(module_annotation)
        background_size = len(background_annotation)
        
        standard_contingency_table = [
                                [module_count, background_count - module_count], 
                                [module_size - module_count, background_size - module_size - (background_count - module_count)]
                            ]
        
        # The -1 and +1 make this more conservative (see explanation from the DAVID database: 
        # https://david.ncifcrf.gov/helps/functional_annotation.html#geneenrich)
        conservative_contingency_table = [
                                [module_count - 1, background_count - module_count + 1], 
                                [module_size - module_count, background_size - module_size - (background_count - module_count)]
                            ]
        
        
        odds, p_standard = st.fisher_exact(standard_contingency_table, 'greater')
        odds, p_conservative = st.fisher_exact(conservative_contingency_table, 'greater')
        
        p_reasonable = np.mean([p_standard, p_conservative])
        
        bonferroni  = p_reasonable * len(module_term_dict)

        fold_enrichment = (module_count/module_size) / (background_count/background_size)

        if bonferroni <= 0.05:
            
            ps.append(p_reasonable)
            bs.append(bonferroni)
            folds.append(fold_enrichment)
            terms.append(t)
            
#         else:
#             ps.append('')
#             bs.append('')
#             folds.append('')
#             terms.append('')
            
    return ps, bs, folds, terms
            
            

In [5]:
def get_GO_info(go_term):
    
    r = requests.get(f'https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/{go_term}/complete', 'html5lib')
    
    go_info = json.loads(r.text)
    
    name = go_info['results'][0]['name']
    
    definition = go_info['results'][0]['definition']['text']
    
    obsolete = go_info['results'][0]['isObsolete']
    
    return name, definition, obsolete

In [6]:
def write_enrichment(lldf, phases, out_prefix, term_column):
    
    with open(f'../enrichment/{out_prefix}_enrichment.txt', 'w') as fl:
    
        for m in tqdm.tqdm(sorted(lldf[f'leiden_label_{phases}'].unique())):

            # print(f'Module {m}')
            ps, bs, folds, terms = enrichment_analysis(m, lldf, phases, complete_annot, term_column)

            fl.write(f'Module {m}\n')
            for b, fold, t in zip(bs, folds, terms):
                
                if term_column == 'COG_category':
                    
                    definition = COG_dict[t]
                    t = f'{t}: {definition}'
                
                elif term_column == 'GOs':
                    name, definition, obsolete = get_GO_info(t)
                    if obsolete:
                        t = f'{name.capitalize()} ({t}): {definition} (obsolete)'
                    else:
                        t = f'{name.capitalize()} ({t}): {definition}'
                        
                elif term_column.split('_')[0] == 'KEGG':
                    dfs = pd.read_html(f'https://www.genome.jp/entry/{t}')
                    
                    data = dfs[4]
                    entry = data.loc[data[0] == 'Entry'][1].values[0]
                    name = data.loc[data[0] == 'Name'][1].values[0]
                    
                    t = f'{t}: {name}'
                        
                # print(f"{t}\nFold-enrichment: {fold:.02f}\nBonferroni-corrected p-value: {b:.02e}\n\n")

                fl.write(f'{t}\nFold-enrichment: {fold:.02f}\nBonferroni-corrected p-value: {b:.02e}\n\n')

In [18]:
def get_KEGG_info(term):
    try:
        dfs = pd.read_html(f'https://www.genome.jp/entry/{term}')
    except:
        return "NA"

    data = dfs[4]
    entry = data.loc[data[0] == 'Entry'][1].values[0]
    name = data.loc[data[0] == 'Name'][1].values[0]
    
    return name

In [8]:
def get_enrichment_df(lldf, phases, background_annotation, term_columns=['COG_category', 'GOs', 'KEGG_ko'], outfile=None):
    
    module_dfs = []
    
    for m in tqdm.tqdm(sorted(lldf[f'leiden_label_{phases}'].unique())):
        
        term_dfs = []

        for tc in term_columns:
        
            ps, bs, folds, terms = enrichment_analysis(m, lldf, phases, background_annotation, tc)

            # fl.write(f'Module {m}\n')
            
            info = []

            if tc == 'GOs':

                for t in terms:
                    name, definition, obsolete = get_GO_info(t)
                    if obsolete:
                        info.append(f'{name.capitalize()}: {definition} (obsolete)')
                    else:
                        info.append(f'{name.capitalize()}: {definition}')
                        
            elif tc == 'COG_category':
                for t in terms:
                    info.append(COG_dict[t])
                                
            elif tc == 'KEGG_ko':
                for t in terms:
                    info.append(get_KEGG_info(t))
                    
            term_df = pd.DataFrame({'module': [int(m)]*len(terms),
                                    'term': terms,
                                    'info': info,
                                    'fold-change': folds,
                                    'bonferroni': bs})
            
            term_dfs.append(term_df)
            
        module_df = pd.concat(term_dfs)
        
        module_dfs.append(module_df)
        
    all_enrichment_df = pd.concat(module_dfs)
    
    if outfile:
        all_enrichment_df.to_csv(outfile, index=False)
    
    return all_enrichment_df
        
        
                        
            
                    
            
        
#         for b, fold, t in zip(bs, folds, terms):

#             if term_column == 'GOs':
#                 name, definition, obsolete = get_GO_info(t)
#                 if obsolete:
#                     t = f'{name.capitalize()} ({t}): {definition} (obsolete)'
#                 else:
#                     t = f'{name.capitalize()} ({t}): {definition}'

#             if term_column.split('_')[0] == 'KEGG':
#                 dfs = pd.read_html(f'https://www.genome.jp/entry/{t}')

#                 data = dfs[4]
#                 entry = data.loc[data[0] == 'Entry'][1].values[0]
#                 name = data.loc[data[0] == 'Name'][1].values[0]

#                 t = f'{t}: {name}'

In [9]:
complete_annotation = pd.read_csv('../eggnog/complete_eggnog_annotation.csv')

In [10]:
lldf = pd.read_csv('../embedding/leiden_label_df_round_1.csv')

In [11]:
lldf_nn2 = pd.read_csv('../embedding/test_nn2_leiden_label_df_round_1.csv')

In [16]:
%pdb

Automatic pdb calling has been turned ON


In [19]:
enrich_full_nn2 = get_enrichment_df(lldf_nn2, 'full', complete_annotation, outfile='./test_nn2_full_enrichment.csv')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2517/2517 [17:09<00:00,  2.44it/s]


In [63]:
enrich_full = get_enrichment_df(lldf, 'full', complete_annotation, outfile='./full_enrichment_leiden_round_1.csv')

100%|███████████████████████████████████████████████████████████████████████████████████| 64/64 [05:39<00:00,  5.31s/it]


In [64]:
enrich_sex = get_enrichment_df(lldf, 'sex', complete_annotation, outfile='./sex_enrichment_leiden_round_1.csv')

100%|███████████████████████████████████████████████████████████████████████████████████| 59/59 [03:36<00:00,  3.67s/it]


In [65]:
enrich_veg = get_enrichment_df(lldf, 'veg', complete_annotation, outfile='./veg_enrichment_leiden_round_1.csv')

100%|███████████████████████████████████████████████████████████████████████████████████| 58/58 [05:22<00:00,  5.56s/it]


In [50]:
enrich['module'].unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 24., 26., 27., 28.,
       29., 30., 31., 32., 33., 35., 36., 38., 39., 41., 42., 45., 46.,
       47., 50., 54., 56., 60., 61., 63.])

In [ ]:
write_enrichment(leiden_label_df_round_1, 'full', 'full_leiden_GOs', 'GOs')

 23%|███████████████████▍                                                               | 15/64 [02:10<00:57,  1.18s/it]

In [ ]:
write_enrichment(consensus_lldf_full_3, 'full', 'consensus_1_full_leiden_GOs', 'GOs')